# Issue Trend Analysis and Issue Tracking

----------------------------------------------------

### Imports & Statics

In [1]:
import csv, pickle
import utils
from utils import Corpus, Issue, Extractor, IssueModel, EventModel
import os, sys
import warnings

warnings.filterwarnings('ignore')

years = [2015, 2016, 2017]
num_issues = 50
num_events = 50
num_keywords = 10

----------------------------------------------------

## Part 0: Preprocessing

### Clean articles: lemmatize, remove stopwords (Already Done)
**_Caution!_ Involves multiprocessing**

In [2]:
# utils.clean_articles()

### Detect Entities: IBM Watson NLU (Already Done)
**_Caution!_ Involves multiprocessing**

In [3]:
# utils.build_watson()

## Part 1: Issue Trend Analysis

#### Initialize Corpuses

In [4]:
corpus = {}
for year in years:
    corpus[year] = Corpus(year=year-2015)

#### Build Corpuses: Load cleaned articles, build phrasers, dictionary, and BOWs

In [5]:
for year in years:
    print("Corpus "+str(year)+":")
    corpus[year].build_corpus()
    print("Corpus "+str(year)+" Done\n")


Corpus 2015:
building corpus...
collecting articles...
building phrasers...
bigram train finished! 7.40 seconds
trigram train finished! 12.16 seconds
building dictionary...
building bag of words...


100%|██████████| 7155/7155 [00:03<00:00, 2038.94it/s]


Corpus 2015 Done

Corpus 2016:
building corpus...
collecting articles...
building phrasers...
bigram train finished! 7.98 seconds
trigram train finished! 12.99 seconds
building dictionary...
building bag of words...


100%|██████████| 7480/7480 [00:03<00:00, 1881.52it/s]


Corpus 2016 Done

Corpus 2017:
building corpus...
collecting articles...
building phrasers...
bigram train finished! 8.76 seconds
trigram train finished! 14.80 seconds
building dictionary...
building bag of words...


100%|██████████| 9117/9117 [00:04<00:00, 2159.72it/s]


Corpus 2017 Done



#### Extract Keywords from each Article using tf-ifd

In [6]:
for year in years:
    print("Corpus "+str(year)+":")
    corpus[year].build_tfidf()
    corpus[year].extractor = Extractor(corpus[year])
    corpus[year].extractor.extract(k=num_keywords)
    print("Corpus "+str(year)+" Done\n")

Corpus 2015:
building tf-idf model...
tfidf finished! 0.07 seconds
extracting keywords...


100%|██████████| 7155/7155 [00:04<00:00, 1682.06it/s]


Corpus 2015 Done

Corpus 2016:
building tf-idf model...
tfidf finished! 0.07 seconds
extracting keywords...


100%|██████████| 7480/7480 [00:04<00:00, 1680.44it/s]


Corpus 2016 Done

Corpus 2017:
building tf-idf model...
tfidf finished! 0.08 seconds
extracting keywords...


100%|██████████| 9117/9117 [00:05<00:00, 1712.40it/s]

Corpus 2017 Done



#### Save Corpuses

In [7]:
utils.save(corpus, filename='corpus_ready.bin')

saving corpus_ready.bin...
corpus_ready.bin saved


#### Load corpus

In [2]:
corpus = utils.load(filename='corpus_ready.bin')

loading corpus_ready.bin...
corpus_ready.bin loaded


#### Build LDA model, cluster articles into issues

In [3]:
for year in years:
    print("Corpus "+str(year)+":")
    corpus[year].build_lda(num_topics=num_issues)
    corpus[year].issue_model = IssueModel(corpus=corpus[year], model=corpus[year].lda)
    corpus[year].issue_model.build_issues()
    print("Corpus "+str(year)+" Done\n")

Corpus 2015:
building LDA model...
LDA finished! 8.57 seconds
building issues...


100%|██████████| 7155/7155 [00:05<00:00, 1238.60it/s]


extracting keywords...


100%|██████████| 50/50 [00:00<00:00, 2583.46it/s]


Corpus 2015 Done

Corpus 2016:
building LDA model...
LDA finished! 9.25 seconds
building issues...


...

Corpus 2016 Done

Corpus 2017:
building LDA model...
LDA finished! 10.00 seconds
building issues...


100%|██████████| 9117/9117 [00:07<00:00, 1292.78it/s]


extracting keywords...


100%|██████████| 50/50 [00:00<00:00, 998.79it/s]

Corpus 2017 Done



#### Save Corpuses

In [4]:
utils.save(corpus, filename='corpus_done.bin')

saving corpus_done.bin...
corpus_done.bin saved


#### Init Issues (for Part 2)

In [5]:
issues = []
for year in years:
    issue_model = corpus[year].issue_model
    top_issue_id = issue_model.sorted_issues[0][0]
    issues.append(Issue(articles=issue_model.issues[top_issue_id], keywords=issue_model.keywords[top_issue_id]))

#### Save Issues (for Part 2)

In [6]:
utils.save(issues, filename='issues_init.bin')

saving issues_init.bin...
issues_init.bin saved


----------------------------------------------------

### Show Results

#### Load corpus

In [2]:
corpus = utils.load(filename='corpus_done.bin')

loading corpus_done.bin...
corpus_done.bin loaded


#### Select year to show

In [3]:
show_year = 2016

#### Show top trending issues  

In [4]:
corpus[show_year].issue_model.show_top_issues()

ID:   5 Score: 1109.61 N:  539 Keywords:  Saenuri, bill, impeachment, Moon, Ahn, Chung, Minjoo, Minjoo_Party, constituency, Opposition_party
ID:  14 Score: 946.36 N:  275 Keywords:  N._Korea, sanction, N_K, S._Korea, human_right, Obama, US, NK, resolution, White_House
ID:   1 Score: 767.23 N:  187 Keywords:  Trump, comfort_woman, Japan, foundation, S._Korea, sex_slavery, Iran, victim, sexual_slavery, sex_slave
ID:  19 Score: 647.80 N:  103 Keywords:  S._Korea, deploy, N_K, nuclear_armament, extended_deterrence, nuclear_weapon, Taurus_missile, military_GPS, exercise, deterrence
ID:  45 Score: 593.15 N:  153 Keywords:  N_K, N._Korea, submarine, site, nuke_test, SLBM_test, SLBM, drill, nuclear_warhead, provocation
ID:   7 Score: 588.85 N:   99 Keywords:  complex, N_K, park, S._Korea, nuclear_envoy, Yun, N._Korea, nuke_test, Hwang, talk
ID:  28 Score: 524.26 N:   50 Keywords:  murder, Patterson, defendant, stab, student, arrest, baby, suspect, fortress, Hwaseong_Fortress
ID:  47 Score: 514

#### Show Articles from Top Issues 

In [5]:
corpus[show_year].issue_model.show_issues(k=5)

ID:   5 Score: 1109.61 N:  539 Keywords:  Saenuri, bill, impeachment, Moon, Ahn, Chung, Minjoo, Minjoo_Party, constituency, Opposition_party
	 0 	 Former leader quits opposition party
	 1 	 [Newsmaker] Park’s lame duck deadline looms
	 2 	 Constitutional reform debate resurfaces
	 3 	 General elections mired in uncertainty without constituencies
	 4 	 Park asks political parties to embrace reform
ID:  14 Score: 946.36 N:  275 Keywords:  N._Korea, sanction, N_K, S._Korea, human_right, Obama, US, NK, resolution, White_House
	 0 	 Quake in North Korea suspected to be 'explosion': report
	 1 	 North Korea announces successful test of hydrogen bomb
	 2 	 China party paper urges North Korea to change 'nuclear path'
	 3 	 Park, Obama talk over N.K. nuke test
	 4 	 Park, Obama agree to closely work together to adopt strong U.N. sanctions against North
ID:   1 Score: 767.23 N:  187 Keywords:  Trump, comfort_woman, Japan, foundation, S._Korea, sex_slavery, Iran, victim, sexual_slavery, sex_slave

#### View LDA Model

In [6]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [8]:
pyLDAvis.gensim.prepare(corpus[show_year].lda, corpus[show_year].get_bows(), corpus[show_year].dict)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.041106  0.211230       1        1  10.581757
14    -0.204319 -0.015433       2        1   6.709829
1     -0.068711  0.080572       3        1   5.046021
19    -0.143959 -0.044118       4        1   4.234733
45    -0.188128 -0.032307       5        1   3.657989
47     0.100641 -0.075179       6        1   3.495347
44    -0.035624  0.023321       7        1   3.236398
37     0.128267 -0.026543       8        1   3.143909
7     -0.129101 -0.004141       9        1   3.110485
28     0.105281 -0.052408      10        1   2.874161
46    -0.120483 -0.013653      11        1   2.635077
11    -0.133701 -0.017885      12        1   2.534039
40     0.101334 -0.000671      13        1   2.406733
39     0.083416 -0.037911      14        1   2.378371
0      0.120659  0.090734      15        1   2.356038
25    -0.189534 -0.044794      16        1   2.168996
33     0.133382 -0.075912      17        1   1.978166
38     0.110236 -0.063264      18        1   1.964928
8      0.099205 -0.101457      19        1   1.924994
13     0.059811 -0.035594      20        1   1.729733
24     0.088932  0.138458      21        1   1.722846
20    -0.057929  0.106298      22        1   1.711893
49     0.080060 -0.057892      23        1   1.598891
41     0.105595 -0.058042      24        1   1.584110
6     -0.083074 -0.047089      25        1   1.432657
15    -0.051472  0.010604      26        1   1.400451
35    -0.073495  0.031989      27        1   1.389371
29     0.035278  0.118114      28        1   1.382589
21     0.043274  0.130647      29        1   1.337908
42     0.050519 -0.003271      30        1   1.242420
48     0.105481 -0.096419      31        1   1.220276
17     0.064395  0.009591      32        1   1.155865
4     -0.101020 -0.026282      33        1   1.138483
27     0.006139 -0.056225      34        1   1.111949
12     0.142194  0.005345      35        1   1.089011
16    -0.099037 -0.052337      36        1   1.082887
30     0.115026 -0.052299      37        1   1.054027
43    -0.087610 -0.077308      38        1   1.034458
26     0.036168  0.101027      39        1   1.004668
10     0.110238  0.002865      40        1   0.910108
36     0.019798 -0.038334      41        1   0.896783
22    -0.011352  0.034560      42        1   0.795051
18    -0.084583  0.020830      43        1   0.763216
2      0.001111  0.061221      44        1   0.707891
31     0.024062  0.003588      45        1   0.674791
23    -0.133475 -0.015634      46        1   0.625109
9     -0.032134 -0.048324      47        1   0.494020
3     -0.035090  0.001604      48        1   0.474712
34    -0.055714  0.061405      49        1   0.454534
32     0.007936  0.026722      50        1   0.341326, topic_info=      Category          Freq                   Term         Total  loglift  \
term                                                                         
554    Default   7784.000000                   Park   7784.000000  30.0000   
16     Default  10898.000000            North_Korea  10898.000000  29.0000   
115    Default   6842.000000                  North   6842.000000  28.0000   
631    Default   4124.000000                percent   4124.000000  27.0000   
433    Default   4517.000000                  China   4517.000000  26.0000   
33285  Default   1118.000000            impeachment   1118.000000  25.0000   
2879   Default   4288.000000                     US   4288.000000  24.0000   
116    Default   3583.000000           North_Korean   3583.000000  23.0000   
1763   Default   2793.000000                   Choi   2793.000000  22.0000   
20856  Default   1450.000000                     NK   1450.000000  21.0000   
1547   Default   4277.000000              president   4277.000000  20.0000   
4268   Default   1376.000000                  THAAD   1376.000000  19.0000   
112    Default   6638.000000                  Korea   663

In [7]:
# issues_lda_data = pyLDAvis.gensim.prepare(corpus[show_year].lda, corpus[show_year].get_bows(), corpus[show_year].dict)
# utils.save(issues_lda_data, 'issues_lda_data.bin')

saving issues_lda_data.bin...
issues_lda_data.bin saved


In [28]:
# pyLDAvis.show(data=data, ip='143.248.137.26', port=9000)

----------------------------------------------------

## Part 2: Issue Tracking

### Imports & Statics

In [2]:
import csv, pickle
import utils
from utils import Corpus, Issue, Extractor, IssueModel, EventModel
import os, sys
import warnings

warnings.filterwarnings('ignore')

years = [2015, 2016, 2017]
num_issues = 50
num_events = 50
num_keywords = 10

#### Load Issues

In [3]:
issues = utils.load('issues_init.bin')

loading issues_init.bin...
issues_init.bin loaded


#### Build Issues

In [4]:
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    issue.build_issue()
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
building issue...
building phrasers...
bigram train finished! 0.30 seconds
trigram train finished! 0.48 seconds
building dictionary...
building bag of words...


100%|██████████| 351/351 [00:00<00:00, 1848.43it/s]


Issue 1 Done

Issue 2:
building issue...
building phrasers...
bigram train finished! 0.79 seconds
trigram train finished! 1.19 seconds
building dictionary...
building bag of words...


100%|██████████| 539/539 [00:00<00:00, 1718.45it/s]


Issue 2 Done

Issue 3:
building issue...
building phrasers...
bigram train finished! 0.94 seconds
trigram train finished! 1.41 seconds
building dictionary...
building bag of words...


100%|██████████| 744/744 [00:00<00:00, 1739.96it/s]

Issue 3 Done



#### Extract keywords from each Article using tf-idf

In [5]:
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    issue.build_tfidf()
    issue.extractor = Extractor(issue)
    issue.extractor.extract(k=num_keywords)
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
building tf-idf model...
tfidf finished! 0.01 seconds
extracting keywords...


100%|██████████| 351/351 [00:00<00:00, 2054.35it/s]


Issue 1 Done

Issue 2:
building tf-idf model...
tfidf finished! 0.01 seconds
extracting keywords...


100%|██████████| 539/539 [00:00<00:00, 1287.19it/s]


Issue 2 Done

Issue 3:
building tf-idf model...
tfidf finished! 0.01 seconds
extracting keywords...


100%|██████████| 744/744 [00:00<00:00, 1573.53it/s]

Issue 3 Done



#### Save Issues

In [6]:
utils.save(issues, filename='issues_ready.bin')

saving issues_ready.bin...
issues_ready.bin saved


#### Load Issues

In [94]:
issues = utils.load(filename='issues_ready.bin')

loading issues_ready.bin...
issues_ready.bin loaded


#### Build LDA model, cluster articles into events

In [95]:
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    issue.build_lda(num_topics=num_events)
    issue.event_model = EventModel(issue=issue, model=issue.lda)
    issue.event_model.build_events(threshold=0.5)
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
building LDA model...
LDA finished! 0.67 seconds
building events...


100%|██████████| 351/351 [00:00<00:00, 1124.75it/s]


extracting keywords...


100%|██████████| 50/50 [00:00<00:00, 8466.84it/s]


Issue 1 Done

Issue 2:
building LDA model...
LDA finished! 1.20 seconds
building events...


...

Issue 2 Done

Issue 3:
building LDA model...
LDA finished! 1.51 seconds
building events...


100%|██████████| 744/744 [00:00<00:00, 1097.80it/s]


extracting keywords...


100%|██████████| 50/50 [00:00<00:00, 7536.39it/s]

Issue 3 Done



#### Divide events into set of independent events

In [96]:
threshold=[0.40, 0.40, 0.40]
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    issue.event_model.build_independents(threshold=threshold[i])
    issue.event_model.filter_events(k=5)
    issue.event_model.build_event_times()
    issue.event_model.build_sorted_independents()
    issue.event_model.build_event_details()
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
building event times...


100%|██████████| 5/5 [00:00<00:00, 1769.60it/s]


building event details...


100%|██████████| 5/5 [00:00<00:00, 845.76it/s]


Issue 1 Done

Issue 2:
building event times...


...

Issue 2 Done

Issue 3:
building event times...


100%|██████████| 5/5 [00:00<00:00, 1089.09it/s]


building event details...


100%|██████████| 5/5 [00:00<00:00, 902.31it/s]

Issue 3 Done



In [97]:
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    print(issue.event_model.sorted_independents)
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
[[28], [44], [18], [26], [19]]
Issue 1 Done

Issue 2:
[[49], [44, 18], [9], [31]]
Issue 2 Done

Issue 3:
[[0], [1], [42, 44], [33]]
Issue 3 Done



#### Save Issues

In [98]:
utils.save(issues, 'issues_done.bin')

saving issues_done.bin...
issues_done.bin saved


----------------------------------------------------

### Show Result

#### Load Issues

In [9]:
issues = utils.load('issues_done.bin')

loading issues_done.bin...
issues_done.bin loaded


#### Select Issue

In [10]:
i = 2

#### Show Issue Keywords

In [101]:
print(', '.join([keyword[0] for keyword in issues[i].keywords]))

Trump, N._Korea, S._Korea, Tillerson, NK, Mattis, THAAD, White_House, dialogue, envoy


#### Show Top Events  

In [102]:
issues[i].event_model.show_top_events()

ID:   1 Score: 137.92 N:   72 Keywords:  Pence, Hwang, Mike_Pence, Vice_President, summit_Xi, proposal, S._Korea, expert, father, US-China
ID:   0 Score:  68.38 N:   36 Keywords:  prepare, Tillerson, Xi, totally, Americans, offer_talk, envoy, President_Xi, Hong, special_envoy
ID:  33 Score:  60.47 N:   36 Keywords:  Mattis, S._Korea, Song, Military, Nauert, State_Department, US_senator, Corker, Bob_Corker, senator
ID:  44 Score:  54.11 N:   37 Keywords:  S._Korea, n't_think, Mattis, Song_Mattis, envoy, Kang, North_Koreans, visit_DMZ, Korean, diplomatic_effort
ID:  42 Score:  53.76 N:   29 Keywords:  S._Korea, Choi, transparency, fix, professor, scandal, defense_spending, unity_N., Hwang, N._Korean
ID:  40 Score:  49.06 N:   27 Keywords:  NK_provocation, Kang, Korea-US_alliance, Tillerson, Chung, channel, Russia, FTA, Yun, objective
ID:  19 Score:  40.05 N:   26 Keywords:  Moon_Abe, level, Mattis, South_Korean_leader, increase_pressure, Shinsuke_Sugiyama, envoy, S._Korea, project, trila

#### Show Articles from Top Events 

In [103]:
issues[i].event_model.show_events(k=5)

ID:   1 Score: 137.92 N:   72 Keywords:  Pence, Hwang, Mike_Pence, Vice_President, summit_Xi, proposal, S._Korea, expert, father, US-China
	 0 	 Korea mulls disclosing defense spending to deflect US' calls for burden sharing
	 1 	 [HERALD INTERVIEW] ‘US-China clash could reset inter-Korean ties’
	 2 	 Trump says ‘100 percent’ with Seoul on NK
	 3 	 Tillerson says Korea already paying 'large amounts' for US troops
	 4 	 Trump reaffirms US security commitment to Japan after N.K. missile launch
ID:   0 Score:  68.38 N:   36 Keywords:  prepare, Tillerson, Xi, totally, Americans, offer_talk, envoy, President_Xi, Hong, special_envoy
	 0 	 Tillerson urges China to 'use all available tools' to rein in N. Korea
	 1 	 N. Korea, THAAD key topics for Trump's summit with Xi: White House
	 2 	 Trump again calls NK problem ‘mess’
	 3 	 Trump to host China's Xi at Florida resort next week
	 4 	 [Herald Interview] ‘US not afraid to strike North Korea in war’
ID:  33 Score:  60.47 N:   36 Keywords:  Mat

#### Show Issue Summary

In [104]:
issues[i].event_model.show_issue_summary(num_entities=10)

Issue Keywords: 
	 Trump, N._Korea, S._Korea, Tillerson, NK, Mattis, THAAD, White_House, dialogue, envoy

Events: 
	 0
	 1
	 42 -> 44
	 33


Event ID:   0
Event Keywords: 
	 prepare, Tillerson, Xi, totally, Americans, offer_talk, envoy, President_Xi, Hong, special_envoy
Time: 2017-08-01
Entities: 
	North Korea, LOCATION
	US, LOCATION
	Donald Trump, PERSON
	China, LOCATION
	South Korea, LOCATION
	Pyongyang, LOCATION
	President Donald Trump, PERSON
	President, JOBTITLE
	president, JOBTITLE
	Beijing, LOCATION

Event ID:   1
Event Keywords: 
	 Pence, Hwang, Mike_Pence, Vice_President, summit_Xi, proposal, S._Korea, expert, father, US-China
Time: 2017-06-25
Entities: 
	North Korea, LOCATION
	South Korea, LOCATION
	US, LOCATION
	Seoul, LOCATION
	Donald Trump, PERSON
	China, LOCATION
	Pyongyang, LOCATION
	President Donald Trump, PERSON
	President, JOBTITLE
	Washington, LOCATION

Event ID:  42
Event Keywords: 
	 S._Korea, Choi, transparency, fix, professor, scandal, defense_spending, unity_N.,

#### View LDA Model

In [11]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [12]:
pyLDAvis.gensim.prepare(issues[i].lda, issues[i].get_bows(), issues[i].dict)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.022313 -0.006185       1        1  15.574217
0      0.032539  0.008115       2        1   7.033676
40     0.019794 -0.001462       3        1   5.342602
33     0.044533 -0.003463       4        1   5.255217
42     0.025272 -0.020749       5        1   5.241739
44     0.029243 -0.002659       6        1   4.992878
45     0.026415  0.027614       7        1   4.007810
19     0.022708  0.006240       8        1   3.959373
10     0.017535 -0.047693       9        1   3.477053
24     0.027267  0.005432      10        1   3.190949
9      0.013224  0.013933      11        1   3.170259
27     0.032012  0.015174      12        1   3.118731
26     0.015454 -0.004672      13        1   2.813293
11     0.018423 -0.015598      14        1   2.412177
48     0.018025 -0.001852      15        1   2.005103
21     0.018172 -0.019067      16        1   1.795410
41     0.024982 -0.014358      17        1   1.705915
7      0.015574  0.000188      18        1   1.622583
30     0.012804  0.011790      19        1   1.527205
47     0.019368 -0.006091      20        1   1.521829
20    -0.000675  0.033611      21        1   1.510465
15     0.018048  0.004487      22        1   1.487296
35     0.021063  0.002472      23        1   1.460956
16     0.013669  0.015873      24        1   1.431952
17     0.011153 -0.022472      25        1   1.392569
2      0.010922 -0.001032      26        1   1.217409
13     0.013492  0.006578      27        1   1.195571
32     0.018959  0.024355      28        1   1.167039
12     0.009358 -0.007147      29        1   1.144970
6      0.005494  0.018208      30        1   0.876253
38     0.019346  0.006305      31        1   0.839025
22     0.011911 -0.002387      32        1   0.731831
31     0.003310 -0.005242      33        1   0.624996
49    -0.033181 -0.028650      34        1   0.610412
29    -0.011124 -0.029324      35        1   0.526277
39    -0.012236 -0.018242      36        1   0.482175
37    -0.013335  0.028204      37        1   0.477074
5     -0.004591 -0.037040      38        1   0.415155
34    -0.020183 -0.037040      39        1   0.389449
46    -0.013587 -0.004152      40        1   0.378855
36    -0.007998  0.043277      41        1   0.374387
3     -0.007304  0.018084      42        1   0.373019
28    -0.069794 -0.057587      43        1   0.248662
43    -0.034776  0.012409      44        1   0.217618
14    -0.055304 -0.000662      45        1   0.176959
18    -0.057233  0.050270      46        1   0.163225
4     -0.062957  0.014992      47        1   0.112682
23    -0.102128  0.002204      48        1   0.095836
8     -0.069598  0.004362      49        1   0.077391
25    -0.036379  0.020648      50        1   0.032476, topic_info=     Category         Freq              Term        Total  loglift  logprob
term                                                                       
108   Default  4454.000000                US  4454.000000  30.0000  30.0000
60    Default  3606.000000       North_Korea  3606.000000  29.0000  29.0000
105   Default  2429.000000             Trump  2429.000000  28.0000  28.0000
13    Default  1710.000000             China  1710.000000  27.0000  27.0000
89    Default  1606.000000       South_Korea  1606.000000  26.0000  26.0000
4876  Default   903.000000              Moon   903.000000  25.0000  25.0000
565   Default  1229.000000              talk  1229.000000  24.0000  24.0000
59    Default  1495.000000             North  1495.000000  23.0000  23.0000
140   Default  1191.000000              also  1191.000000  22.0000  22.0000
77    Default  1009.000000             Seoul  1009.000000  21.0000  21.0000
166   Default   767.000000              call   767.000000  20.0000  20.0000
1143  Default   995.000000          N._Korea   995.000000  19.0000  19.0000
350   Default   657.000000             issue   657.000000  18.0000  18.0000
74    Def

In [54]:
# events_lda_data = pyLDAvis.gensim.prepare(issues[i].lda, issues[i].get_bows(), issues[i].dict)
# utils.save(events_lda_data, 'events_lda_data.bin')

saving events_lda_data.bin...
events_lda_data.bin saved


In [37]:
# pyLDAvis.show(data=data, ip='143.248.137.26', port=9000)